*Practical Data Science 21/22*
# Programming Assignment 2 - Predicting Video Game Sales

In this programming assignment you need to apply your new (or refreshed) machine learning knowledge. You will need to create a modeling pipeline training and evaluating a machine learning model build on several numeric as well as categorical features

## Introduction and Dataset

You are provided with a dataset containing a list of video games with sales greater than 100.000 copies. Your task is to build a model predicting the yearly global sales (column ``Global_Sales``) of a video game leveraging the available features.

To help you get started, the following blocks of code import the dataset using pandas: 

In [1]:
import pandas as pd

In [2]:
data_path = 'https://raw.githubusercontent.com/NikoStein/pds_data/main/data/video_game_sales.csv'
game_sales_data = pd.read_csv(data_path)
game_sales_data = game_sales_data[game_sales_data.Name.isna() == False]
game_sales_data.head()

,Name,Platform,Year_of_Release,Genre,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,82.53,76.0,51.0,8.0,322.0,E
1,Super Mario Bros.,NES,1985.0,Platform,40.24,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,35.52,82.0,73.0,8.3,709.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,32.77,80.0,73.0,8.0,192.0,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,31.37,NaN,NaN,NaN,NaN,NaN


## Splitting the Dataset

Before you can get started training a machine learning model you will have to split the dataframe into features and the target variable (try to use as many features as possible):

In [3]:
game_sales_data.set_index('Name', inplace=True)
game_sales_data.columns

Index(['Platform', 'Year_of_Release', 'Genre', 'Global_Sales', 'Critic_Score',
       'Critic_Count', 'User_Score', 'User_Count', 'Rating'],
      dtype='object')

In [4]:
y = game_sales_data['Global_Sales']
X = game_sales_data.drop('Global_Sales', axis=1)
print(y.head())
print(X.head())

Name
Wii Sports                  82.53
Super Mario Bros.           40.24
Mario Kart Wii              35.52
Wii Sports Resort           32.77
Pokemon Red/Pokemon Blue    31.37
Name: Global_Sales, dtype: float64
                         Platform  Year_of_Release         Genre  \
Name                                                               
Wii Sports                    Wii           2006.0        Sports   
Super Mario Bros.             NES           1985.0      Platform   
Mario Kart Wii                Wii           2008.0        Racing   
Wii Sports Resort             Wii           2009.0        Sports   
Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing   

                          Critic_Score  Critic_Count  User_Score  User_Count  \
Name                                                                           
Wii Sports                        76.0          51.0         8.0       322.0   
Super Mario Bros.                  NaN           NaN         NaN         

Next, you will have to create a train-test split in order to be able to evaluate your models. Use 80\% of the data for training and 20\% for evaluation (take a look at the sklearn [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to identify the relevant parameters):

In [5]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X, y, 
                                                  train_size=0.8, 
                                                  random_state = 0)

## Removing missing values
If you inspect your training data you will find that some of the variables have missing values. Use the ``SimpleImputer`` to replace missing values in numerical columns with the column mean and missing values in categorical columns with the most frequent value (take a look at the SimpleImputer [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) to identify the relevant parameters). You can decide if you want to use the simple or the advanced imputation strategy (or just try both).

In [6]:
from sklearn.impute import SimpleImputer

In [7]:
train_X.dtypes

Platform            object
Year_of_Release    float64
Genre               object
Critic_Score       float64
Critic_Count       float64
User_Score         float64
User_Count         float64
Rating              object
dtype: object

In [8]:
num_cols = [col for col in train_X.columns if train_X[col].dtype == 'float64']
cat_cols = [col for col in train_X.columns if train_X[col].dtype == 'object']

In [10]:
num_imputer = SimpleImputer(strategy='mean')

train_X_num_imputed = pd.DataFrame(num_imputer.fit_transform(train_X[num_cols]), 
                                   columns=num_cols, index=train_X.index)
val_X_num_imputed = pd.DataFrame(num_imputer.transform(val_X[num_cols]), 
                                   columns=num_cols, index=val_X.index)

cat_imputer = SimpleImputer(strategy='most_frequent')

train_X_cat_imputed = pd.DataFrame(cat_imputer.fit_transform(train_X[cat_cols]), 
                                   columns=cat_cols, index=train_X.index)
val_X_cat_imputed = pd.DataFrame(cat_imputer.transform(val_X[cat_cols]), 
                                   columns=cat_cols, index=val_X.index)

## Encoding categorical variables

Prior to training your model you will have to encode the categorical variables. Inspect all categorical variables and use the ``LabelEncoder`` or the ``OneHotEncoder`` where appropriate. Remember that you have to combine the numerical as well as the label encoded and the one hot encoded dataframes at the end.

In [11]:
for cat in cat_cols:
    print("{}: {}".format(cat, game_sales_data[cat].nunique()))

Platform: 31
Genre: 12
Rating: 8


In [12]:
game_sales_data['Rating'].value_counts()

E       3989
T       2958
M       1561
E10+    1419
EC         8
K-A        3
RP         3
AO         1
Name: Rating, dtype: int64

In [14]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [15]:
#see e.g. https://en.wikipedia.org/wiki/Entertainment_Software_Rating_Board for correct order
#see e.g. https://stackoverflow.com/questions/72170947/how-to-use-ordinalencoder-to-set-custom-order for more explanation
ordinal_encoder = OrdinalEncoder(categories=[['EC','E','K-A','E10+','T', 'M', 'AO', 'RP']])

train_X_cat_label = pd.DataFrame(ordinal_encoder.fit_transform(train_X_cat_imputed[["Rating"]]),
                                 columns=["Rating"], 
                                  index=train_X_cat_imputed.index)
val_X_cat_label = pd.DataFrame(ordinal_encoder.transform(val_X_cat_imputed[["Rating"]]),
                                 columns=["Rating"], 
                                 index=val_X_cat_imputed.index)

In [16]:
ohe_encoder = OneHotEncoder(sparse=False)
train_X_cat_ohe = pd.DataFrame(ohe_encoder.fit_transform(train_X_cat_imputed[["Platform", 'Genre']]),
                                 index=train_X_cat_imputed.index, columns=ohe_encoder.get_feature_names_out())
val_X_cat_ohe = pd.DataFrame(ohe_encoder.transform(val_X_cat_imputed[["Platform", 'Genre']]),
                                 index=val_X_cat_imputed.index, columns=ohe_encoder.get_feature_names_out())

In [17]:
train_X = pd.concat([train_X_num_imputed, train_X_cat_label, train_X_cat_ohe], axis=1)
val_X = pd.concat([val_X_num_imputed, val_X_cat_label, val_X_cat_ohe], axis=1)

## Train the Model

Now our dataset should be ready and we can train a predictive model. Train a Decision Tree as well as a Random Forest and compare the in-sample as well as the out-of-sample performance of both models usinge the mean absolute error.

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error# Write your code here

In [19]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model_rf = RandomForestRegressor(n_estimators=100, random_state=1)
    model_rf.fit(X_train, y_train)
    preds_rf = model_rf.predict(X_valid)
    model_dt = DecisionTreeRegressor(random_state=1)
    model_dt.fit(X_train, y_train)
    preds_dt = model_dt.predict(X_valid)
    return mean_absolute_error(y_valid, preds_rf), mean_absolute_error(y_valid, preds_dt)

In [20]:
oos_rf, oos_dt = score_dataset(train_X, val_X, train_y, val_y)
is_rf, is_dt = score_dataset(train_X, train_X, train_y, train_y)

In [21]:
print('Out-of-sample\nRandom Forest: {}\nDecicion Tree" {}'.format(oos_rf, oos_dt))
print('------------------------------')
print('In-sample\nRandom Forest: {}\nDecicion Tree" {}'.format(is_rf, is_dt))

Out-of-sample
Random Forest: 0.4334695197218959
Decicion Tree" 0.5181498358720199
------------------------------
In-sample
Random Forest: 0.22931049440241147
Decicion Tree" 0.12641105283767942


### Use a Pipeline

In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [23]:
train_X, val_X, train_y, val_y = train_test_split(X, y, 
                                                  train_size=0.8, 
                                                  random_state = 0)

In [24]:
# Preprocessing numerical columns
numerical_transformer = SimpleImputer(strategy='mean')

## Preprocessing categorical columns

# Ordinal Encoder
categorical_transformer_ordinal = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(categories=[['EC','E','K-A','E10+','T', 'M', 'AO', 'RP']])) 
])

# One hot encoder
categorical_transformer_ohe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) 
])

# Bundle the preprocessors
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, num_cols),
    ('cat_ordinal', categorical_transformer_ordinal, ['Rating']),
    ('cat_ohe', categorical_transformer_ohe, ["Platform", "Genre"])
])

# Create model
model = RandomForestRegressor(n_estimators=100, random_state=1)


# Bundle preprocessing and modeling code in a pipeline
complete_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Preprocess the raw training data and fit the model
complete_pipeline.fit(train_X, train_y)

# Preprocess the raw validation data and make predictions
preds = complete_pipeline.predict(val_X)

# Evaluate the model
score = mean_absolute_error(val_y, preds)
print("MAE using the complete pipeline: {}".format(score))

MAE using the complete pipeline: 0.43269293521817614


## Improve the Model

Having successfully trained a model, your next task is to improve its performance. Try different advanced feature engineering techniques and see if they are able to improve your model.  

### Catboost Encoding

In [25]:
#!pip install category_encoders
from category_encoders import CatBoostEncoder

In [26]:
train_X, val_X, train_y, val_y = train_test_split(X, y, 
                                                  train_size=0.8, 
                                                  random_state = 0)

In [27]:
num_cols = [col for col in train_X.columns if train_X[col].dtype == 'float64']
cat_cols = [col for col in train_X.columns if train_X[col].dtype == 'object']

num_imputer = SimpleImputer(strategy='mean')

train_X_num_imputed = pd.DataFrame(num_imputer.fit_transform(train_X[num_cols]), 
                                   columns=num_cols, index=train_X.index)
val_X_num_imputed = pd.DataFrame(num_imputer.transform(val_X[num_cols]), 
                                   columns=num_cols, index=val_X.index)

cat_imputer = SimpleImputer(strategy='most_frequent')

train_X_cat_imputed = pd.DataFrame(cat_imputer.fit_transform(train_X[cat_cols]), 
                                   columns=cat_cols, index=train_X.index)
val_X_cat_imputed = pd.DataFrame(cat_imputer.transform(val_X[cat_cols]), 
                                   columns=cat_cols, index=val_X.index)

In [28]:
catboost_encoder = CatBoostEncoder()

train_X_catboostenc = catboost_encoder.fit_transform(train_X_cat_imputed, train_y)
val_X_catboostenc = catboost_encoder.transform(val_X_cat_imputed)

In [29]:
train_X = pd.concat([train_X_num_imputed, train_X_catboostenc], axis=1)
val_X = pd.concat([val_X_num_imputed, val_X_catboostenc], axis=1)

In [30]:
oos_rf, oos_dt = score_dataset(train_X, val_X, train_y, val_y)
is_rf, is_dt = score_dataset(train_X, train_X, train_y, train_y)

In [31]:
print('Out-of-sample\nRandom Forest: {}\nDecicion Tree" {}'.format(oos_rf, oos_dt))
print('------------------------------')
print('In-sample\nRandom Forest: {}\nDecicion Tree" {}'.format(is_rf, is_dt))

Out-of-sample
Random Forest: 0.5166585876720526
Decicion Tree" 0.5329772591262717
------------------------------
In-sample
Random Forest: 0.17344332310914937
Decicion Tree" 4.983420474116061e-20


### Advanced Imputation

In [32]:
train_X, val_X, train_y, val_y = train_test_split(X, y, 
                                                  train_size=0.8, 
                                                  random_state = 0)

# Find all columns with missing values:
cols_with_missing = train_X.columns.values[train_X.isna().sum() > 0]

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    train_X[col + '_was_missing'] = train_X[col].isnull()
    val_X[col + '_was_missing'] = val_X[col].isnull()
    
    
num_cols = [col for col in train_X.columns if train_X[col].dtype == 'float64']
cat_cols = [col for col in train_X.columns if train_X[col].dtype == 'object']
bool_cols = [col for col in train_X.columns if train_X[col].dtype == 'bool']
    
# Imputation
num_imputer = SimpleImputer(strategy='mean')
train_X_num_imputed = pd.DataFrame(num_imputer.fit_transform(train_X[num_cols]),
                              columns=num_cols, index = train_X.index)
val_X_num_imputed = pd.DataFrame(num_imputer.transform(val_X[num_cols]),
                            columns=num_cols, index = val_X.index)

cat_imputer = SimpleImputer(strategy='most_frequent')
train_X_cat_imputed = pd.DataFrame(cat_imputer.fit_transform(train_X[cat_cols]), 
                                   columns=cat_cols, index=train_X.index)
val_X_cat_imputed = pd.DataFrame(cat_imputer.transform(val_X[cat_cols]), 
                                   columns=cat_cols, index=val_X.index)

train_X = pd.concat([train_X_num_imputed, train_X_cat_imputed, train_X[bool_cols]], axis=1)
val_X = pd.concat([val_X_num_imputed, val_X_cat_imputed, val_X[bool_cols]], axis=1)

# Encodings
ordinal_encoder = OrdinalEncoder(categories=[['EC','E','K-A','E10+','T', 'M', 'AO', 'RP']])
train_X_cat_label = pd.DataFrame(ordinal_encoder.fit_transform(train_X_cat_imputed[["Rating"]]),
                                 columns=["Rating"], 
                                 index=train_X_cat_imputed.index)
val_X_cat_label = pd.DataFrame(ordinal_encoder.transform(val_X_cat_imputed[["Rating"]]),
                                 columns=["Rating"], 
                                 index=val_X_cat_imputed.index)


ohe_encoder = OneHotEncoder(sparse=False)
train_X_cat_ohe = pd.DataFrame(ohe_encoder.fit_transform(train_X_cat_imputed[["Platform", 'Genre']]),
                                 index=train_X_cat_imputed.index, columns=ohe_encoder.get_feature_names_out())
val_X_cat_ohe = pd.DataFrame(ohe_encoder.transform(val_X_cat_imputed[["Platform", 'Genre']]),
                                 index=val_X_cat_imputed.index, columns=ohe_encoder.get_feature_names_out())

train_X = pd.concat([train_X_num_imputed, train_X_cat_label, train_X_cat_ohe], axis=1)
val_X = pd.concat([val_X_num_imputed, val_X_cat_label, val_X_cat_ohe], axis=1)

In [33]:
oos_rf, oos_dt = score_dataset(train_X, val_X, train_y, val_y)
is_rf, is_dt = score_dataset(train_X, train_X, train_y, train_y)

print('Out-of-sample\nRandom Forest: {}\nDecicion Tree" {}'.format(oos_rf, oos_dt))
print('------------------------------')
print('In-sample\nRandom Forest: {}\nDecicion Tree" {}'.format(is_rf, is_dt))

Out-of-sample
Random Forest: 0.4334695197218959
Decicion Tree" 0.5181498358720199
------------------------------
In-sample
Random Forest: 0.22931049440241147
Decicion Tree" 0.12641105283767942
